# Saving and Loading Models

TensorFlow에서 모형을 저장하고 불러오는 방법을 알아본다.

## Import Resources

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


In [3]:
tf.__version__

'2.7.0'

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
print('Using:')
print('TensorFlow version:', tf.__version__)
print('tf.keras version:', tf.keras.__version__)
print('Running on GPU' if tf.test.is_gpu_available() else 'GPU device not found. Running on CPU')

Using:
TensorFlow version: 2.7.0
tf.keras version: 2.7.0
GPU device not found. Running on CPU


## Load the Dataset

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [7]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [8]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

## Build and Train the Model



In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 788us/step - loss: 2.2771 - accuracy: 0.3060
Epoch 2/5
1875/1875 [==============================] - 1s 775us/step - loss: 2.3024 - accuracy: 0.2674
Epoch 3/5
1875/1875 [==============================] - 1s 785us/step - loss: 2.3024 - accuracy: 0.2674
Epoch 4/5
1875/1875 [==============================] - 1s 780us/step - loss: 2.3024 - accuracy: 0.2674
Epoch 5/5
1875/1875 [==============================] - 1s 777us/step - loss: 2.3024 - accuracy: 0.2674


In [12]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 2.3026 - accuracy: 0.2560 - 279ms/epoch - 891us/step

Test accuracy: 0.25600001215934753


## Saving and Loading Models

TensorFlow 에서는 모형을 2가지 형태로 저장할 수 있다. 하나는 TensorFlow의 SavedModel format 이고 다른 하나는 HDF5 files format으로 Keras models type 이다.

### Saving and Loading Models in HDF5 Format

 `.save(filepath)` method를 사용하여 모형을 저장한다. 예를 들면 다음과 같다.

```python
my_model.save('./test_model.h5')
```

파일의 확장자는 반드시 `.h5` 으로 한다. 

 HDF5 file 은 다음과 같은 내용을 저장한다.:

* model의 구조.
* model의 weight values
* model의 훈련과 관련된 설정 `compile` method의 parameter값.
* optimizer에 대한 정보


기본적으로 `.save()` method는 에러 없이 기존의 파일을 덮어쓰기 한다.`overwrite=False` 을 설정하면 파일 덮어쓰기 전에 prompt한다.

In [13]:
t = time.time()

saved_keras_model_filepath = './{}.h5'.format(int(t))

model.save(saved_keras_model_filepath)

일단 저장된 모형은 `tf.keras.models.load_model(filepath)` 을 통해 불러들인다. 

In [14]:
reloaded_keras_model = tf.keras.models.load_model(saved_keras_model_filepath)

reloaded_keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


`reloaded_keras_model`은 원래 `model` 과 같은 복사본이 된다. 다음과 같은 명령어로 Prediction 을 수행할 수 있다.

In [15]:
prediction = model.predict(test_images)

### Saving and Loading TensorFlow SavedModels

`model.save('my_model')`을 호출하면 다음을 포함하는 my_model 폴더를 생성한다.

```
$ ls my_model
assets  saved_model.pb  variables
```

모델 아키텍처 및 훈련 구성(옵티마이저, 손실 및 메트릭 포함)은 saved_model.pb에 저장된다. 가중치는 variables/ 디렉토리에 저장된다.


In [16]:
model.save("my_model")

In [17]:
reconstructed_model = tf.keras.models.load_model("my_model")

In [18]:
reconstructed_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Saving Models During Training


훈련중에 validation loss는 훈련이 잘 진행되는지 확인할 수 있는 지표 역할을 한다. overfitting이 일어나지 않는 최적 validation loss의 모형을 저장할 수 있다. 

`tf.keras` 에서 다음과 같은 callback을 사용한다:

```python
tf.keras.callbacks.ModelCheckpoint('./best_model.h5', monitor='val_loss', save_best_only=True)
```

위의 callback은 매 epoch마다 Keras HDF5 file로 저장한다. `save_best_only=True` argument을 주면, 가장 최적의 모형만을 저장한다.

In [19]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Stop training when there is no improvement in the validation loss for 10 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [20]:
# Save the Model with the lowest validation loss
save_best = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',
                                               monitor='val_loss',
                                               save_best_only=True)

In [21]:
history = model.fit(train_images, train_labels,
                    epochs = 100,
                    validation_split=0.3,
                    callbacks=[early_stopping, save_best])

Epoch 1/100
1313/1313 [==============================] - 2s 2ms/step - loss: 0.5202 - accuracy: 0.8142 - val_loss: 0.3998 - val_accuracy: 0.8563
Epoch 2/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.3845 - accuracy: 0.8585 - val_loss: 0.3787 - val_accuracy: 0.8583
Epoch 3/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.3406 - accuracy: 0.8744 - val_loss: 0.3669 - val_accuracy: 0.8663
Epoch 4/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.3201 - accuracy: 0.8830 - val_loss: 0.3779 - val_accuracy: 0.8598
Epoch 5/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.3002 - accuracy: 0.8892 - val_loss: 0.3366 - val_accuracy: 0.8788
Epoch 6/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.2814 - accuracy: 0.8923 - val_loss: 0.3392 - val_accuracy: 0.8748
Epoch 7/100
1313/1313 [==============================] - 2s 1ms/step - loss: 0.2701 - accuracy: 0.8987 - val_loss: 0.3361 - val_ac